## Documentation

To read more about the ingest pipeline, checkout the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/ingest.html).



## Connect to ElasticSearch

In [ ]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

## Create the pipeline

In [ ]:
from pprint import pprint

response = es.ingest.put_pipeline(
    id='lowercase_pipeline',
    description='This pipeline transforms the text to lowercase',
    processors=[
        {
            "lowercase": {
                "field": "text"
            }
        }
    ]
)
pprint(response.body)

## Get the pipeline

In [ ]:
response = es.ingest.get_pipeline(id='lowercase_pipeline')
pprint(response.body)

## Delete a pipeline

In [ ]:
response = es.ingest.delete_pipeline(id='lowercase_pipeline')
pprint(response.body)

## Simulate a pipeline

The simulate method allows you to give the pipeline fake data just to test if it is working or not. This is usually done before applying the pipeline to your real index and data.

In [ ]:
response = es.ingest.put_pipeline(
    id='lowercase_pipeline',
    description='This pipeline transforms the text to lowercase',
    processors=[
        {
            "lowercase": {
                "field": "text"
            }
        }
    ]
)
pprint(response.body)

Inside the docs list, we are providing some test data. After executing the cell, nothing will be indexed. You will just get back how the documents will look like after the transformation.

In [ ]:
response = es.ingest.simulate(
    id='lowercase_pipeline',
    docs=[
        {
            "_index": "my_index",
            "_id": "1",
            "_source": {
                "text": "HELLO WORLD"
            }
        }
    ]
)
pprint(response.body)

## Use the pipeline

Let's read the data and make the text uppercased to see if the `lowercase_pipeline` will be executed before indexing the documents.

In [ ]:
import json

dummy_data = json.load(open("../data/dummy_data.json"))
for i, document in enumerate(dummy_data):
    uppercased_text = document['text'].upper()
    document['text'] = uppercased_text
    dummy_data[i] = document

dummy_data

In [ ]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

Now, we pass the `lowercase_pipeline` to the bulk method. It will perform the transformations before indexing the documents.

In [ ]:
operations = []
for document in dummy_data:
    operations.append({'index': {'_index': 'my_index'}})
    operations.append(document)

response = es.bulk(operations=operations, pipeline='lowercase_pipeline')
pprint(response.body)

After indexing the documents, we can see that the `text` field for all documents has been lowercased. This indicates the pipeline did run with no issues.

In [ ]:
response = es.search(index='my_index')
hits = response.body['hits']['hits']

for hit in hits:
    print(hit['_source'])

## Pipeline failure

### 1. Not handling the failure

In this scenario, we don’t handle failures with `ignore_failure` or `on_failure`. Instead, the pipeline will raise an exception, halting execution of any further processes, and the document will not be indexed.

In [ ]:
response = es.ingest.put_pipeline(
    id='pipeline_1',
    description='Pipeline with multiple transformations',
    processors=[
        {
            "lowercase": {
                "field": "text",
            }
        },
        {
            "set": {
                "field": "text",
                "value": "CHANGED BY PIPELINE",
            }
        },
    ]
)
pprint(response.body)

In [ ]:
document = {
    'title': 'Sample Title 4',
    'created_on': '2024-09-25',
}

response = es.index(
    index='my_index',
    pipeline='pipeline_1',
    body=document
)
pprint(response.body)

### 2. Handling the failure

To handle the failures, we use `ignore_failure` or define an `on_failure` block. With `ignore_failure`, the pipeline will skip over the failed step and continue executing subsequent processes without interrupting the flow, allowing other documents to be indexed.

Alternatively, with `on_failure`, we can specify custom error-handling steps, such as logging the error, retrying, or sending notifications, ensuring the pipeline proceeds even if one step encounters an issue.

In [ ]:
response = es.ingest.put_pipeline(
    id='pipeline_2',
    description='Pipeline with multiple transformations, handling and ignoring failures',
    processors=[
        {
            "lowercase": {
                "field": "text",
                "on_failure": [
                    {
                        "set": {
                            "field": "text",
                            "value": "FAILED TO LOWERCASE",
                            "ignore_failure": True,
                        }
                    }
                ]
            }
        },
        {
            "set": {
                "field": "new_field",
                "value": "ADDED BY PIPELINE",
                "ignore_failure": True,
            }
        },
    ]
)
pprint(response.body)

In [ ]:
document = {
    'title': 'Sample Title 4',
    'created_on': '2024-09-25',
}

response = es.index(
    index='my_index',
    pipeline='pipeline_2',
    body=document
)
pprint(response.body)

In [ ]:
response = es.search(index='my_index')
hits = response.body['hits']['hits']
for hit in hits:
    print(hit['_source'])